In [19]:
!python -m spacy download es_core_news_sm

In [21]:
from pymongo import MongoClient
import wikipedia
import spacy
import openai
from tqdm import tqdm
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
# Get wikipedia page
wikipedia.set_lang("es")
page = wikipedia.page("Sevilla")

In [3]:
# Chunk page into sentences using spacy
nlp = spacy.load("es_core_news_sm")
docs = nlp(page.content)
sentences = [s for s in docs.sents]
len(sentences)

619

In [4]:
# Set up function to create embeddings
# Code snippet copied from https://www.mongodb.com/products/platform/atlas-vector-search
openai.api_key = os.getenv("OPENAI_API_KEY")

model = "text-embedding-ada-002"

def generate_embedding(text: str) -> list[float]:
	resp = openai.Embedding.create(
		input=[text], 
		model=model)
	return resp["data"][0]["embedding"] 

In [5]:
# Set up mongodb connection
client = MongoClient(os.environ['MONGODB_URI'])
db = client["vectordb-test"]
collection = db['sevilla']

In [31]:
# What was the vector dimensions again?
documents[0]['vector'].__len__()

1536

In [10]:
documents = [
    {
        "text": sentence.text,
        "vector": generate_embedding(sentence.text),
        "source": page.url,   
    } for sentence in tqdm(sentences)
]













































































































































































































































































































































































































































































































































































































































100%|██████████| 619/619 [03:05<00:00,  3.34it/s]


In [12]:
documents[0:1]

[{'text': 'Sevilla es un municipio y una ciudad de España, capital de la provincia homónima y de Andalucía.[6]\u200b Contaba con 681 998 habitantes en 2022,[7]\u200b por lo que es la ciudad más poblada de Andalucía, la cuarta de España[8]\u200b después de Madrid, Barcelona y Valencia y la 26.ª de la Unión Europea.',
  'vector': [-0.009113910607993603,
   -0.015544665046036243,
   -0.0021619554609060287,
   0.0008282510098069906,
   -0.023921651765704155,
   0.02484123222529888,
   -0.019752053543925285,
   0.03506997227668762,
   -0.006392964161932468,
   -0.04459328576922417,
   -0.007753436919301748,
   0.02096136286854744,
   0.023266609758138657,
   0.0022280896082520485,
   1.3212059457146097e-05,
   -0.0056780856102705,
   0.014486519619822502,
   -0.028469160199165344,
   0.02462708204984665,
   0.0025335662066936493,
   0.001788770197890699,
   0.007218065671622753,
   0.008314002305269241,
   -0.012974882498383522,
   -0.007274752017110586,
   0.0035177047830075026,
   0.01830

In [13]:
collection.insert_many(documents)

In [ ]:
#https://www.mongodb.com/docs/atlas/atlas-search/field-types/knn-vector/